# Wrangling Data from Laboratory Reports - Google Colab Session

> *Let me remind the reader that Abalone Labs and Gottagetta Life are mythical entities. The names, identifiers, and test results in the AbLab print files were all fabricated from random values.*


When you run one of the following Colab code cells for the first time, Google will warn you that this is not a Google notebook. Choose the option "Run Anyway".

For example, **Click on the arrow in the next cell** to determine what version of Python Google is using, then choose the option "Run Anyway".  

*one can also run a code cell by placing the cursor in the cell and keying "Shift-Enter"*

In [1]:
!python --version

Python 3.12.12


Python 3.12.12 was released in 10/2025 as a security fix for version 3.12 which was first released in 10/2023. Since the current stable version of Python is 3.14, one may occasionally find that the latest Python expressions may not work in Colab. See https://docs.python.org/release/3.12.12/ for version specific documentation.

*Colab can also be configured to run R code.*

---

I am using GitHub to store this Notebook and its associated data files. If you got this far, you already have a Colab copy of my notebook. Now you need to do next is to **click the arrow in the next cell** to make a temporary copy of my entire GitHub repository including the data files that we will want to use.

In [2]:
!git clone "https://github.com/dowes48/LabReports"

Cloning into 'LabReports'...
remote: Enumerating objects: 519, done.
remote: Counting objects: 100% (519/519), done.
remote: Compressing objects: 100% (185/185), done.
remote: Total 519 (delta 397), reused 428 (delta 332), pack-reused 0 (from 0)
Receiving objects: 100% (519/519), 36.49 MiB | 24.91 MiB/s, done.
Resolving deltas: 100% (397/397), done.


After running the previous cell, click on the directory icon located in the panel to the left of this Notebook page. You will see a new directory titled "LabReports". Open the LabReports directory, then the AbLab_Rpts subdirectory. Under its subdirectories, you will find the target files, all with ".prn" file extensions. Double-click to open your choice of .prn files. You will see the file contents in a panel to the right of the page.

---
Note: this notebook is ephemeral as are the cloned repository files. They will be deleted sometime after you have finished using the Colab notebook.


## Overall Strategy

We need a program to visit each of the subdirectories and its sub-sbudirectgories; open each prn file; process the file line by line; store test values for a particular report in a buffer (there are multiple lab reports in most of the prn files); flush the buffer to a pipe delimited text file when a form feed is detected (end of report); and then start processing the next report.

I prefer pipes (|) over commas because they are easier for me to read. A stray comma in processed text will cause havoc in a comma separated values (.csv) file so this is another reason to prefer pipe separated values (.psv).

### First Steps

Let's start by practicing a "walk" through the target directory and its sub-directories. This simple exercise does nothing but verify we have a systematic way to visit each file. The output will be a listing of each directory and each file name in that directory.

---

**Before running the following code**, place your cursor in the code cell and locate the cell's context menu. All Colab cells have a context menu that can be located at the top right corner of the cell. "Context" means the menu choices are specific to the cell type. Choose **Explain Code** either from the 3 dot drop down menu or by clicking the pencil icon. Gemini, Google's branded AI, **may** generate a thorough explanation of the code (displayed in the right pane). I say **may** because sometimes Gemini returns a lazy summary without details.

---

Now run the cell. After you are finished viewing the verbose ouput from this code cell, you may want to clewar it by choosing "**Clear Output**" from the context menu.

In [ ]:
TARGETDIR = '/content/LabReports/AbLab_Rpts'
import os

print(f"Walking through directory: {os.path.abspath(TARGETDIR)}\n")
# Iterate over the 3-tuple generated by os.walk()
for dirpath, dirnames, filenames in os.walk(TARGETDIR):
    # Print the current directory path
    print(f"Current Directory: {dirpath}")

    # List subdirectories found
    if dirnames:
        print(f"  Subdirectories: {', '.join(dirnames)}")

    # Iterate over files in the current directory
    for filename in filenames:
        # Construct and print the full path of each file
        full_file_path = os.path.join(dirpath, filename)
        print(f"  Found file: {full_file_path}")
    print("-" * 40)


Note the variable "TARGETDIR" above. Since I won't be changing its value, it is essentially a constant. I use all caps for such variables.

**import os** makes all of the **o**perating **s**ystem functions available for the program to use. This includes director walking.

Also note the two ***for*** loops. There are other loops available in Python, but the construct ***for xx in yy*** is the most Pythonic.
The ***os.walk()*** function returns a three-tuple. See Prof Downey's text for an explanation of tuples.

In the ***print*** statements, the ***f*** prefix indicates a formatted string literal and allows for easy interpolation of variable values in the printed string.

---

The above code demonstrated how to traverse the directory tree and touch each file. Now let's open each file and "do something", but keep it simple for now. We will take advantage of the fact that each lab report is followed by a form feed character, ***\f***. Counting form feeds will tell us how many reports there are in the .prn files.


In [ ]:
form_feed = '\f'
form_feed_count = 0
line_count = 0
file_count = 0

def process_line(f_in):
    global line_count, form_feed_count
    for line in f_in:
        line_count += 1
        if form_feed in line:
            form_feed_count += 1

for dirpath, dirnames, filenames in os.walk(TARGETDIR):
    for filename in filenames:
        file_count += 1
        full_file_path = os.path.join(dirpath, filename)
        file_in = open(full_file_path, 'r')
        process_line(file_in)
        file_in.close()

print(f"All {file_count:,} .prn files were opened")
print(f"A total of {line_count:,} lines were searched for a form feed.")
print(f"There are {form_feed_count:,} lab test reports available for processing.")


The previous code cell includes two new features:
  1) a custom function, process_line(), was defined;
  2) the concept of "scope" is introduced.
See Downey for a full explanation of function definitions.

Note that I need to have the "count" variables available at the main level in order to print their values after completion of the processing. In order to modify the variables from within the function definition, I need to declare them as **global** variables inside the function definition. Otherwise, the varibles would be "local" to the function (despite having the same names) and the count increases would never appear at the main level.

Scope will come into play again as we expand the code to incoporate:
*   an output file for collecting lab results
*   a buffering system that uses a Python dictionary to collect results from each lab report instance

Everything is working well, but it is best to proceed incrementally rather than trying to do it all at once. Let's start by narrowing our attention to one of the sub-directories that has only a handful of .prn files. Let's open a file for output, write a short series of column headers to it. Then we will write a small number of values from inside **process_reports(f_in)**. Try to remember to close both input and output files at the end of the program.

Two new Python data types - *lists* and *dictionaries* are introduced in the next code cell. If you are not familiar with these important types, please review chapters 9 and 10 in Downey.



In [9]:
TARGETDIR = r'/content/LabReports/AbLab_Rpts/AbLab_2018-20'
OUTPUTDIR = r'/content/LabReports/Output'

import os
import copy

lab_file_out = open(OUTPUTDIR + r'/labs_output.psv', 'w')
lab_file_out.write('name|sex|ticket|gluc\n')

clean_lab_dict = {'name':"", 'sex':"", 'ticket':"", 'gluc':""}

def process_reports(f_in):
    lab_dict = copy.deepcopy(clean_lab_dict)
    lab_lst = []
    for line in f_in:
        if "\f" not in line:
            if 'NAME:' in line:
                lab_dict['name'] = line[6:].strip()
            if 'DOB/SEX:' in line:
                lab_dict['sex'] = line[21]
            if 'TICKET NUMBER:' in line:
                lab_dict['ticket'] = line[15:25]
            if 'GLUCOSE (MG/DL' in line:
                lab_dict['gluc'] = line[30:40].strip()
        else:
            for k, v in lab_dict.items():
                lab_lst.append(v)
            lab_file_out.write("|".join(lab_lst) + '\n')
            lab_dict = copy.deepcopy(clean_lab_dict)
            lab_lst = []

for dirpath, dirnames, filenames in os.walk(TARGETDIR):
    # Iterate over files in the current directory
    for filename in filenames:
        # Construct and print the full path of each file
        full_file_path = os.path.join(dirpath, filename)
        file_in = open(full_file_path, 'r')
        process_reports(file_in)
        file_in.close()

lab_file_out.close()


In the left pane, under "files/LabReports/Output", double-click the file "labs_output.txt". Its contents will appear in the right pane.

Note Jamison's glucose value, "NVG" (not valid, glycolysis). This can be a result for any of the tests. A similar return is "NVH" (not valid, hemolysis). These will cause headaches when you try to import the file into Excel. Excel will convert a column of compatible strings into their numerical values (which is what you want) unless any one of them is not convertible into a number, e.g. 'NVG' or 'NVH'. While we could skip these instances and just return a blank value, it is better to not throw away information. We will define a function that replaces them with "magic numbers": 9998 for NVG and 9999 for NVH.

Note that we are in the thick of string processing, which is Python's wheelhouse. Every 'line' returned via the expression 'line in f_in' is a string, and the [] indices return slices of the line string. Thus, line[21] returns a single character (sex); line[15:25] returns the 10-degit ticket number; and line[6:] is open-ended on the right meaning it returns the string starting at position 6 through to the end of the line. While this will always contain the individual's NAME, there will usually be a lot of extra space characters. Thus the .strip() METHOD is applied to remove empty spaces.

Review Chapter 8 of Downey with special attention to diagram that shows how Python numbers string positions. The numbers actually apply not to the characters but to the points between the characters. This is a subtle but important point.

The previous code cell is also the first time we've used lists or dictionaries. The lab_dict is an easy way to store lab values temporarily, but then moving those values in a list provides us with the easiest way to output the values while joining them with pipes.

The line:

    for k, v in lab_dict.items()

returns a 2-tuple: a key (e.g. 'NAME') and its value (e.g. "Kareem D Metallo"). We only need the values for our output list.

---

In the next code cell, we will:


1.   define a function to address the "NVG" issue,
2.   calculate the applicant's age from the dob and date collected values,
1.   separate out systolic and diastolic BP values.

TARGETDIR is changed to "AbLab_2022" because that is the first sub-directory that contains files with BP values.

In [7]:
TARGETDIR = r'/content/LabReports/AbLab_Rpts/AbLab_2022'
OUTPUTDIR = r'/content/LabReports/Output'

import os
import copy
from datetime import datetime

clean_lab_dict = {'ticket':"", 'd_coll':"", 'age':"", 'sex':"", 'd_perf':"", 'face':"",
                  'gluc':"", 'fruct':"", 'a1c':"", 'bun':"",'creat':"", 'alkp':"", 'bpsys':"", 'bpdias':""}
lab_file_out = open(OUTPUTDIR + r'/labs_output.psv', 'w')
lab_file_out.write('ticket|d_coll|age|sex|d_perf|face|gluc|fruct|a1c|bun|creat|alkp|bpsys|bpdias\n')

def fixMV(valStr):
    if valStr =="NVG":
        return "9998"
    if valStr=="NVH":
        return "9999"
    return valStr

def process_reports(f_in):
    lab_dict = copy.deepcopy(clean_lab_dict)
    lab_lst = []
    for line in f_in:
        if "\f" not in line:
            if 'TICKET NUMBER:' in line:
                lab_dict['ticket'] = line[15:25]
                lab_dict['d_perf'] = line[55:].strip()
            if 'DOB/SEX:' in line:
                date_dob = datetime.strptime(line[9:19].strip(), "%m/%d/%Y")
                lab_dict['sex'] = line[21]
            if 'SOC SEC NO:' in line:
                lab_dict['d_coll'] = line[55:].strip()
                date_coll = datetime.strptime(lab_dict['d_coll'],"%m/%d/%Y")
                age_days = (date_coll - date_dob).days
                lab_dict['age'] = str(int(age_days/365.25))
            if 'TYPE/AMT:'  in line:
                lab_dict['face'] =  line[22:].strip().replace(r',', '')
            if 'GLUCOSE (MG/DL' in line:
                lab_dict['gluc'] = fixMV(line[30:40].strip())
            if 'FRUCTOSAMINE  ' in line:
                lab_dict['fruct'] = fixMV(line[30:40].strip())
            if 'HB A1C (%)    ' in line:
                lab_dict['a1c'] = fixMV(line[30:40].strip())
            if 'BUN (MG/DL)   ' in line:
                lab_dict['bun'] = fixMV(line[30:40].strip())
            if 'CREATININE (MG' in line:
                lab_dict['creat'] = fixMV(line[30:40].strip())
            if 'ALK. PHOS. (U/' in line:
                lab_dict['alkp'] = fixMV(line[30:40].strip())
            if 'BLOOD PRESSURE' in line:
                t_str = line[30:40].strip()
                bp_list = t_str.split('/')
                lab_dict['bpsys']  = bp_list[0]
                lab_dict['bpdias'] = bp_list[1]
        else:
            for k, v in lab_dict.items():
                lab_lst.append(v)
            lab_file_out.write("|".join(lab_lst) + '\n')
            lab_dict = copy.deepcopy(clean_lab_dict)
            lab_lst = []

for dirpath, dirnames, filenames in os.walk(TARGETDIR):
    # Iterate over files in the current directory
    for filename in filenames:
        # Construct and print the full path of each file
        full_file_path = os.path.join(dirpath, filename)
        file_in = open(full_file_path, 'r')
        process_reports(file_in)
        file_in.close()

lab_file_out.close()


Double-click on labs_output.txt. Now we have 72 lines of results, of which 4 include BP values - they're the ones without a double pipe (||) at the end. The BP processing appears to be correct.

---

This is a good time to address the issue of sensitive data. Note that none of personal identifiers are to be found in the current data output. Such data can be shared with actuaries and underwriters without concern about privacy issues. It may be tempting to proceed this way, ignore the personal identifiers, and just add the rest of the test results to the dataset. However, one should avoid throwing away data. Assuming the identifier data can be kept secure in the medical department, it can be very useful. In the near future, it may be necessary to match lab results with an individual. More importantly, good identifiers will allow for mortality follow-up.

The *ticket number* is a unique identifier (no duplicates). This provides the opportunity to have two datasets: 1) one with test results; 2) a second with personal identifiers that can be used for mortality follow-up. The test dataset can be shared as needed while the second dataset should be kept secure in the medical department. For a study relating test results to mortality, the two datasets can be joined on the ticket number to create one complete dataset.

To that end, we will create two separate dictionaries, two separate output text files, and we will write two different sets of column headers to the output files.

The long list of test name recognition and value capture code is tedious, but not difficult.

In [8]:
TARGETDIR = r'/content/LabReports/AbLab_Rpts'
OUTPUTDIR = r'/content/LabReports/Output'

import os
import copy
from datetime import datetime

clean_ID_dict = {'ticket':"", 'd_coll':"", 'name':"",'dob':"", 'sex':"", 'ssn':"",'zip':""}
ID_file_out = open(OUTPUTDIR + r'/ID_output.psv', 'w')
ID_file_out.write('ticket|d_coll|name|dob|sex|ssn|zip\n')

clean_lab_dict = {'ticket':"", 'd_coll':"", 'age':"", 'sex':"", 'd_perf':"", 'face':"",
              'gluc':"", 'fruct':"", 'a1c':"", 'bun':"",'creat':"", 'alkp':"",
              'bil_t':"", 'ast':"", 'alt':"", 'ggt':"", 'prot':"", 'albu':"",
              'glob':"", 'chol':"", 'hdl':"", 'tch_r':"", 'trigs':"", 'u_pro':"",
              'u_cre':"", 'u_pcr':"", 'bmi':"", 'pulse':"", 'bpsys':"", 'bpdias':""}
lab_file_out = open(OUTPUTDIR + r'/labs_output.psv', 'w')
lab_file_out.write('ticket|d_coll|age|sex|d_perf|face|gluc|fruct|a1c|bun|creat|alkp|bil_t|' +
                   'ast|alt|ggt|prot|albu|glob|chol|hdl|tch_r|trigs|u_pro|u_cre|u_pcr|bmi|' +
                   'pulse|bpsys|bpdias\n')

def fixMV(valStr):
    if valStr =="NVG":
        return "9998"
    if valStr=="NVH":
        return "9999"
    return valStr

def process_reports(f_in):
    lab_dict = copy.deepcopy(clean_lab_dict)
    lab_lst = []
    ID_dict = copy.deepcopy(clean_ID_dict)
    ID_lst = []

    for line in f_in:
        if "\f" not in line:
            if 'NAME:'          in line:
                ID_dict['name'] = line[6:].strip()
            if 'DOB/SEX:'       in line:
                ID_dict['dob'] = line[9:19].strip()
                ID_dict['sex'] = line[21]
                lab_dict['sex'] = line[21]
                ID_dict['zip'] = line[60:].strip()
                date_dob = datetime.strptime(line[9:19].strip(), "%m/%d/%Y")
            if 'SOC SEC NO:'    in line:
                ID_dict['ssn'] = line[12:21]
                ID_dict['d_coll'] = line[55:].strip()
                lab_dict['d_coll'] = line[55:].strip()
                date_coll = datetime.strptime(lab_dict['d_coll'],"%m/%d/%Y")
                age_days = (date_coll - date_dob).days
                lab_dict['age'] = str(int(age_days/365.25))
            if 'TICKET NUMBER:' in line:
                lab_dict['ticket'] = line[15:25]
                ID_dict['ticket'] = line[15:25]
                lab_dict['d_perf'] = line[55:].strip()
            if 'TYPE/AMT:'      in line:
                lab_dict['face'] =  line[22:].strip().replace(r',', '')
            if 'GLUCOSE (MG/DL' in line:
                lab_dict['gluc'] = fixMV(line[30:40].strip())
            if 'FRUCTOSAMINE  ' in line:
                lab_dict['fruct'] = fixMV(line[30:40].strip())
            if 'HB A1C (%)    ' in line:
                lab_dict['a1c'] = fixMV(line[30:40].strip())
            if 'BUN (MG/DL)   ' in line:
                lab_dict['bun'] = fixMV(line[30:40].strip())
            if 'CREATININE (MG' in line:
                lab_dict['creat'] = fixMV(line[30:40].strip())
            if 'ALK. PHOS. (U/' in line:
                lab_dict['alkp'] = fixMV(line[30:40].strip())
            if 'BILI. TOT. (MG' in line:
                lab_dict['bil_t'] = fixMV(line[30:40].strip())
            if 'AST(SGOT) (U/L' in line:
                lab_dict['ast'] =  fixMV(line[30:40].strip())
            if 'ALT(SGPT) (U/L' in line:
                lab_dict['alt'] =  fixMV(line[30:40].strip())
            if 'GGT(GGTP) (U/L' in line:
                lab_dict['ggt'] =  fixMV(line[30:40].strip())
            if 'TOT. PROTEIN (' in line:
                lab_dict['prot'] = fixMV(line[30:40].strip())
            if 'ALBUMIN (G/DL)' in line:
                lab_dict['albu'] = fixMV(line[30:40].strip())
            if 'GLOBULIN (G/DL' in line:
                lab_dict['glob'] = fixMV(line[30:40].strip())
            if 'CHOLESTEROL (M' in line:
                lab_dict['chol'] = fixMV(line[30:40].strip())
            if 'HDL CHOLESTERO' in line:
                lab_dict['hdl'] =  fixMV(line[30:40].strip())
            if 'CHOL/HDL CHOL ' in line:
                lab_dict['tch_r'] = fixMV(line[30:40].strip())
            if 'TRIGLYCERIDES ' in line:
                lab_dict['trigs'] = fixMV(line[30:40].strip())
            if 'URINE PROTEIN ' in line:
                lab_dict['u_pro'] = fixMV(line[30:40].strip())
            if 'URINE CREATINI' in line:
                lab_dict['u_cre'] = fixMV(line[30:40].strip())
            if 'PROT/CREAT RAT' in line:
                lab_dict['u_pcr'] = fixMV(line[30:40].strip())
            if 'BMI           ' in line:
                lab_dict['bmi'] =  fixMV(line[30:40].strip())
            if 'PULSE         ' in line:
                lab_dict['pulse'] = fixMV(line[30:40].strip())
            if 'BLOOD PRESSURE' in line:
                t_str = line[30:40].strip()
                bp_list = t_str.split('/')
                lab_dict['bpsys']  = bp_list[0]
                lab_dict['bpdias'] = bp_list[1]
        else:
            for k, v in lab_dict.items():
                lab_lst.append(v)
            lab_file_out.write("|".join(lab_lst) + '\n')
            lab_dict = copy.deepcopy(clean_lab_dict)
            lab_lst = []

            for k, v in ID_dict.items():
                ID_lst.append(v)
            ID_file_out.write("|".join(ID_lst) + '\n')
            ID_dict = copy.deepcopy(clean_ID_dict)
            ID_lst = []


for dirpath, dirnames, filenames in os.walk(TARGETDIR):
    # Iterate over files in the current directory
    for filename in filenames:
        # Construct and print the full path of each file
        full_file_path = os.path.join(dirpath, filename)
        file_in = open(full_file_path, 'r')
        process_reports(file_in)
        file_in.close()

lab_file_out.close()
ID_file_out.close()

For the first time, the processing is not near-instantaneous - I experienced anywhere from 3 to 9 second times for the code to run. See the green checkmark and runtime value at the upper left position of this pane.

We are done with the Python part, but we still need to move the data into an Excel workbook. **Note** that because of their size, double-clicking these output files will sometimes display them in the right pane, but more often a dialog box will ask you for a download location on your drive.

>
1.   Open a blank Excel workbook.
2.   Open the Output directory in the left pane of this notebook.
1.   This time right-click "labs_output.psv" and choose "Download". Be patient, these are now large files. When the file contents are finally ready, a dialog box will appear for you to specify where to save the file as.
2.   Open the file that is now on your hard drive, key "Ctrl-A" to select All the file contents, followed by "Ctrl-C" to copy the contents to your clipboard.
1.   Paste contents into cell A1 of your Excel file.
2.   Select the Data menu and then "Text to Columns" from the ribbon.
1.   On the first page of the Wizard, choose delimited, and then hit "Next".
2.   Unselect tab, select other, type a pipe character into the box and then choose  "Finish".
1.   Rename tab to "Labs", adjust column widths, freeze top row, and all the other things one does with Excel spreadsheets.

Use a new tab in the Excel workbook to repeat the above steps for "ID_output.psv"

---

Now return to the print article for a discussion of the results.